In [ ]:
! pip install -q simpletransformers

     |████████████████████████████████| 248 kB 13.6 MB/s 
     |████████████████████████████████| 311 kB 78.4 MB/s 
     |████████████████████████████████| 1.7 MB 82.7 MB/s 
     |████████████████████████████████| 1.2 MB 76.2 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 6.8 MB 77.4 MB/s 
     |████████████████████████████████| 9.7 MB 80.8 MB/s 
     |████████████████████████████████| 3.5 MB 89.6 MB/s 
     |████████████████████████████████| 895 kB 78.7 MB/s 
     |████████████████████████████████| 596 kB 82.4 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 180 kB 80.0 MB/s 
     |████████████████████████████████| 144 kB 86.4 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 1.1 MB 78.9 MB/s 
     |████████████████████████████████| 243 kB 92.9 MB/s 
     |████████████████████████████████| 133 kB 88.7 MB/s 
     |██████████████

In [ ]:
import logging

import torch
import pandas as pd
from simpletransformers.ner import NERModel, NERArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
! pip install -q datasets transformers[sentencepiece] simpletransformers

In [ ]:
! wget https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz

In [ ]:
! tar -xvf 'AIFORTHAI-LST20Corpus.tar.gz' # untar

In [ ]:
from datasets import load_dataset
lst20 = load_dataset("lst20", data_dir="/content/LST20_Corpus") 

In [ ]:
lst20

In [ ]:
import pandas as pd #   simple tranformer รับข้อมูลเป็น dataframe
train_df = pd.DataFrame(lst20['train'])
validation_df = pd.DataFrame(lst20['validation'])
test_df = pd.DataFrame(lst20['test'])

In [ ]:
train_df

In [ ]:
# See tags from https://github.com/huggingface/datasets/blob/master/datasets/lst20/lst20.py
_POS_TAGS = ["NN", "VV", "PU", "CC", "PS", "AX", "AV", "FX", "NU", "AJ", "CL", "PR", "NG", "PA", "XX", "IJ"]

In [ ]:
def convert_to_simple_transformer_format(df, field_name, tags):
  sentence_id = []
  words = []
  labels = []

  #Limit at 1000 rows for speed.
  for (idx, r) in df[:].iterrows():
    # print(idx)
    for (i, t) in enumerate(r['tokens']):
      # print(i,t)
      sentence_id.append(idx)
      words.append(t)
      labels.append(tags[r[field_name][i]])

  return pd.DataFrame(
      {"sentence_id": sentence_id, "words": words, "labels": labels} # simple transformer format
  )


In [ ]:
train_ = convert_to_simple_transformer_format(train_df, "pos_tags", _POS_TAGS)

In [ ]:
train_

In [ ]:
validation_ = convert_to_simple_transformer_format(validation_df, "pos_tags", _POS_TAGS)
test_ = convert_to_simple_transformer_format(test_df, "pos_tags", _POS_TAGS)

In [ ]:
# Configure the model
model_args = NERArgs()
model_args.train_batch_size = 32
model_args.evaluate_during_training = True
model_args.overwrite_output_dir = True # True for overwrite
model_args.num_train_epochs = 10
model_args.max_seq_length = 384
model_args.best_model_dir = '/content/drive/MyDrive/POSTAG/BERT_model'

model = NERModel(
    "camembert", "/content/drive/MyDrive/POSTAG/BERT_model", args=model_args, use_cuda=torch.cuda.is_available(), labels=_POS_TAGS
)

# Train the model
#model.train_model(train_, eval_data=validation_)

/usr/local/lib/python3.7/dist-packages/simpletransformers/ner/ner_model.py:379: UserWarning: use_multiprocessing automatically disabled as CamemBERT fails when using multiprocessing for feature conversion.
  "use_multiprocessing automatically disabled as CamemBERT"


In [ ]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["มีชัย _ ฤชุพันธุ์ ยัน ไม่ เคย เป็น ที่ปรึกษา"])
predictions

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'มีชัย': 'NN'},
  {'_': 'PU'},
  {'ฤชุพันธุ์': 'NN'},
  {'ยัน': 'VV'},
  {'ไม่': 'NG'},
  {'เคย': 'AX'},
  {'เป็น': 'VV'},
  {'ที่ปรึกษา': 'NN'}]]

In [ ]:
# Evaluate the model
result, model_outputs, preds_list = model.eval_model(test_)

NameError: ignored

In [ ]:
# read test file
list_word = []
count = 0
with open('/content/pos_test.txt') as f:
    count = count + 1
    contents = f.read()
    list_word.append(contents)
print(list_word)

['อย่างไรก็ตาม\nเครื่อง\nบิน\nแอร์บัส\n\nเอ\n380\nได้\nถึง\nที่\nหมาย\nที่\nท่าอากาศยาน\nเชียงใหม่\nใน\nเวลา\n\n14.50\n\nน.\nโดย\nใช้\nเวลา\nบิน\nประมาณ\n1\nชั่วโมง\n\nมูลนิธิสืบฯ\nพอใจ\nฟื้น\nป่าห้วยขาแข้ง\nมี\nเสือโคร่ง\nมาก\nอันดับ\n2\nโลก\n\n1\n\nกันยายน\n\n2550\n17.31\nน.\n\nประธาน\nมูลนิธิ\nสืบ\nนาคะเสถียร\nพอใจ\nป่า\nห้วยขาแข้ง\nฟื้นตัว\nพบ\nเสือโคร่ง\n2-3\nตัว\nต่อ\n100\nตาราง\nกิโลเมตร\nมาก\nอันดับ\nที่\n\n2\nของ\nโลก\nตั้ง\nเป้า\n5\nปี\nขึ้น\nอันดับ\n1\nแสดง\nถึง\nยัง\nมี\nสัตว์\nและ\nพืช\nชนิด\nอื่น\nๆ\nอุดมสมบูรณ์\n\nวอน\nรัฐ\nเข้าใจ\nปรับ\nนโยบาย\nจับตา\nค้าน\nกฎหมาย\nเปิด\nป่า\n\n(\n1\nก.ย.\n)\nนาง\nรตยา\n\nจันทรเทียร\nประธาน\nมูลนิธิ\nสืบ\nนาคะเสถียร\nให้\nสัมภาษณ์\nถึง\nสถานการณ์\nป่า\nและ\nสัตว์\nป่า\nใน\nป่าห้วยขาแข้ง\nจ.\nอุทัยธานี\nว่า\nจาก\nการ\nวิจัย\nทาง\nวิชาการ\n\nล่าสุด\nพบ\nว่า\nผืน\nป่า\nห้วยขาแข้ง\nซึ่ง\nเป็น\nผืน\nป่า\nขนาด\nใหญ่\nด้าน\nตะวันตก\nเป็น\nมรดก\nทาง\nธรรมชาติ\nของ\nโลก\nพื้นที่\nแรก\nของ\nไทย\nตั้งแต่\nปี\n2534\n\nขณะ\nนี้\nมี\nเสือโคร่ง\nอาศัย

In [ ]:
list_word[0] = list_word[0][:-2]

In [ ]:
print(len(list_word[0]))

461276

In [ ]:
list_word_new =  list_word[0].replace('\n',' ')

In [ ]:
#len(list_word_new)
temp = list_word_new.split(' ')
print(len(temp))

103913


In [ ]:
list_word_new_under = list_word_new.replace('  ',' _ ')

In [ ]:
#list_word_new_under
temp2 = list_word_new_under.split(' ')
print(len(temp2))

103913


In [ ]:
list_word_new_under

'อย่างไรก็ตาม เครื่อง บิน แอร์บัส _ เอ 380 ได้ ถึง ที่ หมาย ที่ ท่าอากาศยาน เชียงใหม่ ใน เวลา _ 14.50 _ น. โดย ใช้ เวลา บิน ประมาณ 1 ชั่วโมง _ มูลนิธิสืบฯ พอใจ ฟื้น ป่าห้วยขาแข้ง มี เสือโคร่ง มาก อันดับ 2 โลก _ 1 _ กันยายน _ 2550 17.31 น. _ ประธาน มูลนิธิ สืบ นาคะเสถียร พอใจ ป่า ห้วยขาแข้ง ฟื้นตัว พบ เสือโคร่ง 2-3 ตัว ต่อ 100 ตาราง กิโลเมตร มาก อันดับ ที่ _ 2 ของ โลก ตั้ง เป้า 5 ปี ขึ้น อันดับ 1 แสดง ถึง ยัง มี สัตว์ และ พืช ชนิด อื่น ๆ อุดมสมบูรณ์ _ วอน รัฐ เข้าใจ ปรับ นโยบาย จับตา ค้าน กฎหมาย เปิด ป่า _ ( 1 ก.ย. ) นาง รตยา _ จันทรเทียร ประธาน มูลนิธิ สืบ นาคะเสถียร ให้ สัมภาษณ์ ถึง สถานการณ์ ป่า และ สัตว์ ป่า ใน ป่าห้วยขาแข้ง จ. อุทัยธานี ว่า จาก การ วิจัย ทาง วิชาการ _ ล่าสุด พบ ว่า ผืน ป่า ห้วยขาแข้ง ซึ่ง เป็น ผืน ป่า ขนาด ใหญ่ ด้าน ตะวันตก เป็น มรดก ทาง ธรรมชาติ ของ โลก พื้นที่ แรก ของ ไทย ตั้งแต่ ปี 2534 _ ขณะ นี้ มี เสือโคร่ง อาศัย อยู่ 2-3 ตัว ต่อ พื้นที่ 100 ตาราง กิโลเมตร _ หรือ รวม มี เสือโคร่ง ประมาณ 80 - 100 ตัว ถือ เป็น อันดับ 2 ของ โลก รอง จาก ผืน ป่า ไซบีเรีย _ แต่ ทาง 

In [ ]:
from tqdm.notebook import tqdm
# split batch of string to list
list_of_sentence = []
subsentence = ''
for i in tqdm(list_word_new_under):
  subsentence = subsentence + i
  if i == '_':
    list_of_sentence.append(subsentence)
    subsentence = ''


  0%|          | 0/478070 [00:00<?, ?it/s]

In [ ]:
# way 2 ##### use this
from tqdm.notebook import tqdm
split_text = list_word_new_under.split(' ')
textarr = []
subsentence = []
sep = ' '
count = 0
# split batch of string to list
for i in tqdm(split_text):
  count = count + 1
  subsentence.append(i)
  if count == 10:
    count = 0
    textarr.append(sep.join(subsentence))
    subsentence = []
textarr.append(sep.join(subsentence))


  0%|          | 0/103913 [00:00<?, ?it/s]

In [ ]:
print(textarr)

In [ ]:
list_of_sentence

In [ ]:
#### warning may cause bug ######
list_of_sentence_new = []
for string in tqdm(list_of_sentence):
  #print(string)
  if not (string.endswith(' _')):
    string = string[:-2] + ' _'
    list_of_sentence_new.append(string)
  else:
    list_of_sentence_new.append(string)

  0%|          | 0/16906 [00:00<?, ?it/s]

In [ ]:
# Prediction
predictions, raw_output = model.predict(textarr)

INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/10392 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1299 [00:00<?, ?it/s]

In [ ]:
len(predictions)

10392

In [ ]:
predictions[0]

[{'อย่างไรก็ตาม': 'CC'},
 {'เครื่อง': 'NN'},
 {'บิน': 'VV'},
 {'แอร์บัส': 'NN'},
 {'_': 'PU'},
 {'เอ': 'NN'},
 {'380': 'NU'},
 {'ได้': 'AX'},
 {'ถึง': 'VV'},
 {'ที่': 'NN'}]

In [ ]:
#predict list
prediction = []
for dict_predict in tqdm(predictions):
  for word in dict_predict:
    #print(word)
    pos = list(word.values())[0]
    #print(pos)
    prediction.append(pos)
    #pos = list(dict_predict.values())[0]
    #prediction.append(pos)

  0%|          | 0/10392 [00:00<?, ?it/s]

In [ ]:
sub_df = pd.read_csv('/content/pos_sample_submission.csv')
#sub_df = sub_df.set_index('Id')

In [ ]:
for i in tqdm(range(len(sub_df))):
  sub_df.loc[i,'Predicted'] = prediction[i]

  0%|          | 0/103913 [00:00<?, ?it/s]

In [ ]:
sub_df.to_csv('./sample_submission.csv', index = False)